In [ ]:
import ee
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

from e_sbae.ee import time_series

### select your EE asset that keeps your points and the point id name

In [ ]:
# put in your EE feture collection
points = ee.FeatureCollection(
    "your_point_collection"
).sort('point_id')

# define the column name fo your unique point identifier
point_id_name = "point_id"

# print out number of total points
points.size().getInfo()

### set your basic configuration

In [ ]:
# start of calibration period (mainly for bfast)
start_calibration = "2017-01-01"  # YYYY-MM-DD format

# Actual period of interest, i.e. monitoring period
start_monitor =     "2018-01-01"  # YYYY-MM-DD format
end_monitor =       "2019-03-01"  # YYYY-MM-DD format

# Directory where output and temp files will go
outdir = 'results'  # goes to module_results/sbae_point_analysis if set to None

# select the bands to extract
bands = ['green', 'red', 'nir', 'swir1', 'swir2', 'ndfi'] #, 'brightness', 'greenness', 'wetness'] # other choices: ndfi, ndmi, mndwi, brightness, greenness, wetness

# select the band for univariate ts-analysis (has to be inside bands list)
ts_band = 'ndfi'

# select the resolution to which the satellite data will be resized.
scale = 100  # in meters (70 m is half ha, relates to FAO forest definition)

### DO NOT CHANGE YET ###
satellite='Landsat'  # this is going to be Surface Reflactance, Collection 2, Tier 1 data only
max_cloud_cover = 75  # in percentage (0-100)

### advanced configuration (you can leave that to defaults)

In [ ]:
# landsat related parameters
lsat_params = {
    'l9':     True,
    'l8':     True,
    'l7':     True,
    'l5':     True,
    'l4':     True,
    'brdf':   True,
    'bands':  bands,
    'max_cc': max_cloud_cover
}

### DO NOT CHANGE ###
### GATHER ALL INFO INTO A DICT #####
config_dict = {
    'work_dir':                         outdir,
    'workers':                          10,
    'max_points_per_chunk':             1500,
    'grid_size_levels':                 [0.5, 0.25, 0.125, 0.075],  # definition of chunk sizes in degrees  
    'lsat_params':                      lsat_params,
    'ts_params': {
        'start_calibration':            start_calibration,
        'start_monitor':                start_monitor,
        'end_monitor':                  end_monitor,
        'point_id':                     point_id_name,
        'bands':                        bands,
        'ts_band':                      ts_band,
        'satellite':                    satellite,
        'scale':                        scale,
        'max_cc':                       max_cloud_cover,
        'outlier_removal':              True,
        'smooth_ts':                    True       
    }
}

### extract the time-series

**Note** that in your tmp folder (home/sepal-user/tmp), you will have a logfile that will be updated, even if you disconnected to your sepal instance in between processing. After processing has finished, the log file will be copied to your results folder, in case processing has gone wrong or is incomplete. The name format of the file is: 

esbae_log_YYMMDD_HHMMSS (YYMMDD_HHMMSS is the date and hour the logfile was created)

In [ ]:
time_series.extract(points, config_dict)